##Task 1. Deploy the exported Cosmetic Inspection anomaly detection solution artifact

In [ ]:
# Compute Engine -> VM Instances

In [ ]:
# Connect to the VM
gcloud compute ssh lab-vm --zone us-east4-a

In [ ]:
# Create variables
export DOCKER_TAG=gcr.io/ql-shared-resources-test/defect_solution@sha256:776fd8c65304ac017f5b9a986a1b8189695b7abbff6aa0e4ef693c46c7122f4c

In [ ]:
export VISERVING_CPU_DOCKER_WITH_MODEL=${DOCKER_TAG}
export HTTP_PORT=8602
export LOCAL_METRIC_PORT=8603

In [ ]:
# Pull the Image
docker pull ${VISERVING_CPU_DOCKER_WITH_MODEL}

In [ ]:
# Docker RUN
docker run -v /secrets:/secrets --rm -d --name "product_inspection" \
--network="host" \
-p ${HTTP_PORT}:8602 \
-p ${LOCAL_METRIC_PORT}:8603 \
-t ${VISERVING_CPU_DOCKER_WITH_MODEL} \
--metric_project_id="${PROJECT_ID}" \
--use_default_credentials=false \
--service_account_credentials_json=/secrets/assembly-usage-reporter.json

In [ ]:
docker container ls

##Task 2. Prepare resources to serve the exported assembly inspection solution artifact

In [ ]:
# Copy the prediction script
gsutil cp gs://cloud-training/gsp895/prediction_script.py .

In [ ]:
# Create a Cloud Storage bucket
export PROJECT_ID=$(gcloud config get-value core/project)
gsutil mb gs://${PROJECT_ID}

In [ ]:
# Copy images to the bucket
gsutil -m cp gs://cloud-training/gsp897/cosmetic-test-data/*.png \
gs://${PROJECT_ID}/cosmetic-test-data/

In [ ]:
# The 2 files
gsutil cp gs://${PROJECT_ID}/cosmetic-test-data/IMG_07703.png .
gsutil cp gs://${PROJECT_ID}/cosmetic-test-data/IMG_0769.png .

In [ ]:
ls

##Task 3. Identify a defective product image

In [ ]:
# Identify a defective product image
python3 ./prediction_script.py --input_image_file=./IMG_07703.png  --port=8602 \
--output_result_file=defective_product.json

##Task 4. Identify a non-defective product

In [ ]:
# Identify a non-defective product
python3 ./prediction_script.py --input_image_file=./IMG_0769.png  --port=8602 \
--output_result_file=non_defective_product.json